In [2]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
from sklearn.cluster import KMeans
import json
import numpy as np
import os
from tqdm import tqdm
import time
import random
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity



In [17]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [18]:
batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# Load pre-trained VGG16 model
vgg16 = torchvision.models.vgg16(pretrained=True)
# Modify first layer with 1 channel
# vgg16.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
# Remove the last fully connected layer to get feature extraction layer
vgg16.features = torch.nn.Sequential(*list(vgg16.features.children())[:-1])

Files already downloaded and verified
Files already downloaded and verified


C:\Users\paul8\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\paul8\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Move model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)


# Extracting Features

In [22]:
# Extract features for each image in the dataset
features = []
i = 0
with torch.no_grad():
    for image, label in trainloader:
        print(i)
        image = image.to(device)
        features.append(vgg16(image).cpu().numpy().flatten())
        #features.append(vgg16(image.unsqueeze(0)).cpu().numpy())
        #labels.append(labels)
        i = i+1


0
1


OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 4.00 GiB total capacity; 2.40 GiB already allocated; 0 bytes free; 3.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
torch.save(features, 'CIFAR10features.pth')

#loaded_tensor = torch.load('fashionmnistfeatures.pth')
